In [ ]:
import numpy as np

from data import Movie, Rating
from recommender import NearestNeighbor

np.random.seed(42)

In [ ]:
movie_database = Movie.load()
rating_database = Rating.load(nrows=10**6)

# User-based Recommendation

In [ ]:
model = NearestNeighbor(proxies=rating_database.data['userId'],
                        targets=rating_database.data['movieId'],
                        ratings=rating_database.data['rating'])

proxy = np.random.choice(model.proxy_encoder.classes_)

## Already Watched

In [ ]:
ratings = rating_database.find(proxy)
movie_database.find(ratings.index).join(ratings).sort_values(by='title')

## Should Watch

In [ ]:
movie_database.find(model.recommend_targets(proxy)).sort_values(by='title')

# Item-based Recommendation

In [ ]:
model = NearestNeighbor(proxies=rating_database.data['movieId'],
                        targets=rating_database.data['userId'],
                        ratings=rating_database.data['rating'])

proxy = np.random.choice(model.proxy_encoder.classes_)

## Already Watched

In [ ]:
movie_database.find([proxy])

## Should Watch

In [ ]:
movie_database.find(model.recommend_proxies(proxy)).sort_values(by='title')